# Stock Screening na Bovespa
- tutorial por @victordhn

> `(desatualizado, out-2023) pacote para baixar tickers do IBOV está com problema; irei atualizar com meu próprio código para baixar os tickers (ver projeto 4).`

### Resumo rápido:
A idéia deste projeto é criar uma lista com as ações que tiveram a melhor performance relativo à um índice e buscar alguns indicadores de mercado da ação. No caso, o índice é ibovespa e performance é medida pelo retorno acumulado de 1 ano.

Para simplificar, usei como indicdores apenas o P/L ratio, o Lucro por Ação, Capitalização de Mercado e o max/min das últimas 52 semanas. Você pode buscar mais indicadores, ou criar condições especiais para rankear as ações - no meu caso, o ranking é unicamente baseado no retorno acima do indicador.

### Libraries utilizadas:
- yfinance
- yahoo_fin
- datetime
- pandas

### Alguns pontos importantes:
- A performance do código na parte de baixar os indicadores é ruim, ele demora para baixar, portanto recomendo utilizar poucas ações na lista final enquanto você estiver testando o modelo. 
- A performance não deverá ser um problema quando você for rodar o modelo apenas 1 vez.
- Não tenho como afirmar que a lista de tickers do yahoo_fin é 100% atualizada. Para ser mais preciso, você deverá usar o outro código de Webscraping de composição de índices que tenho no repositório

In [1]:
import pandas as pd
import yfinance as yf
from yahoo_fin import stock_info as si
import datetime as dt

# para ignorar uns warnings chatos do Jupyter
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Honestamente não sei o quão atualizado é esta lista de tickers do yahoo
tickers = si.tickers_ibovespa()

In [3]:
# ajustando para por o .SA nos tickers (necessário para utilizar biblioteca yfinance)
tickers_sa = []
for ticker in tickers:
    tickers_sa.append(f'{ticker}.SA')
    

In [4]:
# definindo data de início e fim com 1 ano de separação
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [5]:
# definindo o retorno do índice para futuras comparações
ibovespa = yf.download('^BVSP',start,end)
ibovespa['Pct Change']=ibovespa['Adj Close'].pct_change()
ibovespa['Return']=(ibovespa['Pct Change']+1).cumprod()
ibovespa_return = ibovespa['Return'][-1]

[*********************100%***********************]  1 of 1 completed


In [6]:
# baixando dados das ações dos índices e salvando o retorno comparado para cada ação na lista return_list
return_list = []
for ticker in tickers_sa:
    try:
        df = yf.download(ticker,start,end)
        df['Pct Change']=df['Adj Close'].pct_change()
        df['Return']=(df['Pct Change']+1).cumprod()
        stock_return = df['Return'][-1]
        stock_return_compared = round((stock_return/ibovespa_return),2)
        return_list.append(stock_return_compared)
    
    # este try/except é necessário pois o modelo precisa preencher a lista mesmo quando houver falha na busca...
    # ...logo, preencheremos com 0,00 (isso não afeta a busca pelas melhores ações, mas afetará a busca pelas piores)
    except:
        return_list.append(0.00)
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRDT3.SA: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***

In [20]:
# rankeando as ações de acordo com o seu retorno comparado e separando as 10 melhores
best_performers = pd.DataFrame(list(zip(tickers_sa,return_list)),columns=['Ticker','Returns Compared'])
best_performers['Score']=best_performers['Returns Compared'].rank(pct=True)*100
best_performers.sort_values(by=['Score'],ascending=False,inplace=True)
top_10 = best_performers.head(10)
top_10

,Ticker,Returns Compared,Score
25,CYRE3.SA,1.45,100.000000
14,BRSR6.SA,1.44,98.684211
57,PETR4.SA,1.43,97.368421
56,PETR3.SA,1.40,96.052632
30,EMBR3.SA,1.39,94.078947
1,AZUL4.SA,1.39,94.078947
3,BBAS3.SA,1.36,91.447368
69,UGPA3.SA,1.36,91.447368
38,GOLL4.SA,1.33,89.473684
59,RADL3.SA,1.26,87.500000


In [43]:
# código para buscar os indicadores paras as ações no top 10 de retornos anuais agregados

# obs: o gargalo do código não é na importação novamente pelo yfinance, mas sim na busca pelos indicadores...
# ... utilizando o yahoo_fin

final_df = pd.DataFrame(columns=['Ticker','Preço','Retorno Comparado','Min 52 semanas',\
                                 'Max 52 semanas','PL','LPA','Market Cap'])
for ticker in top_10['Ticker']:
    latest_price = si.get_quote_table(ticker)['Previous Close']
    PL = si.get_quote_table(ticker)['PE Ratio (TTM)']
    LPA = si.get_quote_table(ticker)['EPS (TTM)']
    market_cap = si.get_quote_table(ticker)['Market Cap']
    
    df = yf.download(ticker,start,end)
    low_52week = round(min(df['Low'][-(52*5):]),2)
    high_52week = round(max(df['High'][-(52*5):]),2)
    retorno = top_10.loc[top_10['Ticker']==ticker,'Returns Compared'].values[0]
    
    
    final_df = final_df.append({'Ticker':ticker,
                    'Preço':latest_price,
                    'Min 52 semanas':low_52week,
                    'Max 52 semanas':high_52week,
                    'Retorno Comparado': retorno,
                    'PL':PL,
                    'LPA':LPA,
                    'Market Cap':market_cap},ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [44]:
# resultado final
final_df

,Ticker,Preço,Retorno Comparado,Min 52 semanas,Max 52 semanas,PL,LPA,Market Cap
0,CYRE3.SA,19.91,1.45,11.58,21.14,8.77,2.40,7.892B
1,BRSR6.SA,14.90,1.44,9.08,15.20,6.29,2.40,6.117B
2,PETR4.SA,29.65,1.43,20.77,38.39,2.24,13.15,411.82B
3,PETR3.SA,33.31,1.40,23.61,42.08,2.51,13.18,411.819B
4,EMBR3.SA,18.60,1.39,10.75,21.27,NaN,-1.25,13.392B
5,AZUL4.SA,19.14,1.39,6.71,22.33,NaN,-2.07,6.71B
6,BBAS3.SA,49.01,1.36,30.25,51.99,3.86,12.93,142.575B
7,UGPA3.SA,18.45,1.36,11.33,19.39,11.84,1.63,21.027B
8,GOLL4.SA,11.90,1.33,5.03,13.53,NaN,-0.25,38.399B
9,RADL3.SA,28.05,1.26,18.46,29.90,46.70,0.61,48.843B
